# Part I. Ingesting data.

Create a dictionary with movies for each genre.

In [1]:
import numpy as np
import math
import time
import matplotlib.pyplot as plt

### First, read in the similarity matrix. Not all movies have ratings, so we'll ignore the ones that are unrated.

In [2]:
simDataFile = open("mvs.csv", "r")
simData = [ map(float,line.split(',')) for line in simDataFile ]

In [3]:
print len(simData), ' movies have ratings'

10473  movies have ratings


In [4]:
simMat = dict()

for sd in simData:
    simMat[int(sd[0])] = sd[1:] / np.linalg.norm(sd[1:])

In [5]:
np.dot(simMat[1], simMat[4])

0.75932462063717165

In [6]:
len(simMat[1])

25

### Read in the ratings. We will be working only with the top 150 movies. Ratings are in the format - userId,movieId,rating,timestamp

In [7]:
ratingsFile = open("procRatings.csv", "r")

ratings = dict()

for line in ratingsFile:
    data = map(float,line.split(','))
    ratings[int(data[0])] = data[1]
    
ratings[1], ratings[2], ratings[3], ratings[30848]

(3.92123956132, 3.21197680169, 3.15104043973, 3.61520190024)

#### Set the ratings of the 150'th movie as a threshold

In [8]:
minRating = sorted(ratings.values(), reverse=True)[149]

### Now, read in the movies.

In [9]:
movies = dict() # a category -> list_of_movies dict stored as integers

# catID returns the index of a category of type string. catName returns the name of the category given its ID.
catID = dict()
catName = dict()

# same as with cat
movieID = dict()
movieName = dict()

In [10]:
allData = open("movies.csv", "r")

# this first line contains header info
allData.readline()

numMovies = 0
numCategories = 0

while True:
    line = allData.readline()
    
    if line == '':
        break
    
    curMovieID = int(line.split(",", 1)[0])
    if curMovieID not in simMat.keys():
        continue
    if ratings[curMovieID] < minRating:
        continue
    
    curMovieName = (line.split(",", 1)[1]).rsplit(",", 1)[0]
    curCategories = line.rsplit(",", 1)[1].rsplit("\r")[0].split("|")
    
    
    # update catID, catName, movieID, movieName
    movieID[curMovieName] = curMovieID
    movieName[curMovieID] = curMovieName
    
    for cat in curCategories:
        if not (cat in catID):
            catID[cat] = numCategories
            catName[numCategories] = cat
            
            numCategories = numCategories + 1
            
    for cat in curCategories:
        if catID[cat] in movies:
            movies[catID[cat]].append(movieID[curMovieName])
        else:
            movies[catID[cat]] = [movieID[curMovieName]]
    
    numMovies = numMovies + 1

print "we have", numMovies, "movies"

we have 150 movies


In [11]:
movies[catID['Horror']][:10]

[593]

In [12]:
simDist = dict()

for k1 in movieName.keys():
    for k2 in movieName.keys():
        simDist[(k1, k2)] = np.dot(simMat[k1], simMat[k2])

# Part II. Fix l = 15. Increase k

In [13]:
l = 20

print numMovies, numCategories, l

150 19 20


In [14]:
from replacementGreedy import replacementGreedy
from greedysum import gsWrapper
from greedymerge import gmWrapper
from localsearch import lsWrapper

In [15]:
solution = []
runtime = []
calls = []

Kvalues = [1,2,4,5,7]
for k in range(10, l + 5, 5):
    Kvalues.append(k)

for k in Kvalues:
    rg = replacementGreedy(numMovies, numCategories, l, k, simDist, movies)
    ls = lsWrapper(numMovies, numCategories, l, k, 0.2, simDist, movies)
    gs = gsWrapper(numMovies, numCategories, l, k, simDist, movies)
    gm = gmWrapper(numMovies, numCategories, l, k, simDist, movies)
    
    curSol = []
    curRt = []
    curCalls = []
    
    for alg in [rg, ls, gs, gm]:
        start = time.time()
        
        # TODO: edit this out
#         if alg == ls:
#             curRt.append(0)
#             curSol.append(0)
#             curCalls.append(0)
#             continue
            
        algS, algCost, algEvals = alg(movieName.keys())
        
        if alg == rg:
            print 'Replacement Greedy gives cost', algCost
        
        curRt.append(time.time() - start)
        curSol.append(algCost)
        curCalls.append(algEvals)
    
    solution.append(curSol)
    runtime.append(curRt)
    calls.append(curCalls)
    
    print ""
    print "Done for k = ", k
    print "\n\n\n"

Replacement Greedy gives cost 357.380422868
Local search value after initialization =  357.380422868
Intermediate cost at step  0  =  357.380422868
Local Search gives cost =  357.380422868
Greedy Sum gives cost =  355.415968527
Greedy Merge gives cost =  357.380422868
Size of S is  15

Done for k =  1




Replacement Greedy gives cost 362.835285675
Local search value after initialization =  362.738493504
Intermediate cost at step  0  =  362.761977825
Intermediate cost at step  1  =  362.789146852
Intermediate cost at step  2  =  362.807138304
Intermediate cost at step  3  =  362.840346407
Intermediate cost at step  4  =  362.840346407
Local Search gives cost =  362.840346407
Greedy Sum gives cost =  361.542272291
Greedy Merge gives cost =  362.866436008
Size of S is  31

Done for k =  2




Replacement Greedy gives cost 365.279711884
Local search value after initialization =  365.269072768
Intermediate cost at step  0  =  365.292706957
Intermediate cost at step  1  =  365.317892728
Int

In [16]:
# store locally
filename = open('../data/movielens-L.txt', 'w')

print>>filename, 'solution'
for item in solution:
    print>>filename,item[0],item[1],item[2],item[3]
    
print>>filename, 'runtime'
for item in runtime:
    print>>filename,item[0],item[1],item[2],item[3]
    
print>>filename, 'calls'
for item in calls:
    print>>filename,item[0],item[1],item[2],item[3]

In [17]:
plt.clf()

ax = plt.subplot(111)

fs = 17

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('k', fontsize=fs)
ax.set_ylabel('Objective Value', fontsize=fs)

plt.ylim(355,372)
plt.xlim(min(Kvalues), max(Kvalues))

# ax.set_xticks(range(min(Kvalues),max(Kvalues)),1)
xticks = [min(Kvalues)]
for k in range(5, max(Kvalues) + 5, 5):
    xticks.append(k)

ax.set_xticks(xticks)

colors = ['ro', 'cs', 'b^', 'gv']
labelNames = ['Replacement Greedy', 'Local Search', 'Greedy Sum', 'Greedy Merge']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(solution)):
        vals.append(solution[i][ind])
        
    if ind != 0:
        plt.plot(Kvalues, vals, c = colors[ind][0], marker = colors[ind][1], linewidth=2, label = labelNames[ind])
    else:
        plt.plot(Kvalues, vals, 'r--', linewidth=4, label = labelNames[ind])
    
# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='lower right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')

plt.savefig("../../writeup/images/movielens-fixed-l")

plt.close()

# plt.show()

In [18]:
plt.clf()

ax = plt.subplot(111)

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('k', fontsize=fs)
ax.set_ylabel('Log(runtime)', fontsize=fs)

plt.ylim(-10,25)
plt.xlim(min(Kvalues), max(Kvalues))

xticks = [min(Kvalues)]
for l in range(10, max(Kvalues) + 10, 10):
    xticks.append(l)

ax.set_xticks(xticks)

colors = ['ro', 'cs', 'b^', 'gv']
labelNames = ['Replacement Greedy', 'Local Search', 'Greedy Sum', 'Greedy Merge']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(runtime)):
        vals.append(math.log(runtime[i][ind], 2))
        
    if ind != 0:
        plt.plot(Kvalues, vals, c = colors[ind][0], marker = colors[ind][1], linewidth=2, label = labelNames[ind])
    else:
        plt.plot(Kvalues, vals, 'r--', linewidth=4, label = labelNames[ind])
    
# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='upper right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')

plt.savefig("../../writeup/images/movielens-runtime-fixed-l")

plt.close()